In [1]:
import os
import json
import pickle
import requests
import collections
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
with open('data/jeopardy_358974_questions_20190612.pkl', 'rb') as f:
    karl_to_question_id = pickle.load(f).to_dict()['question_id']
with open('data/jeopardy_310326_question_player_pairs_20190612.pkl', 'rb') as f:
    records_df = pickle.load(f)
    
def get_result(card):
    record_id = karl_to_question_id[int(card['question_id'])]
    prob = (records_df[records_df.question_id == record_id]['correct'] / 2 + 0.5).mean()
    return 'correct' if prob > 0.5 else 'wrong'

In [28]:
with open('data/diagnostic_questions.pkl', 'rb') as f:
    cards = pickle.load(f)

for i, card in enumerate(cards):
    cards[i]['user_id'] = 'diagnostic'

requests.post('http://127.0.0.1:8000/api/karl/reset')

shown_cards = []
results = []
while len(shown_cards) < 5:
    r = requests.post('http://127.0.0.1:8000/api/karl/schedule', data=json.dumps(cards))
    r = json.loads(r.text)
    show_card = cards[r['order'][0]]
    show_card['label'] = get_result(show_card)
    shown_cards.append(show_card)
    results.append(1 if show_card['label'] == 'correct' else 0)
    print('{: <40}{: <20}{: <20}{: <20}'.format(
        show_card['answer'],
        show_card['category'],
        show_card['label'],
        np.mean(results)
    ))
    print(r['rationale'])
    print()
    r = requests.post('http://127.0.0.1:8000/api/karl/update', data=json.dumps([show_card]))
    cards = [x for x in cards if x['question_id'] != show_card['question_id']]

Bashar al-Assad                         WORLD LEADERS       wrong               0.0                 
qrep: 0.724
skill: 0.139
category: 0.000
leitner: 0.000
sm2: 0.000
sum: 0.170
user: 0.568
card: 0.429
topic: 17.000

Germany & the Czech Republic            WORLD GEOGRAPHY     correct             0.5                 
qrep: 0.783
skill: 0.054
category: 0.000
leitner: 0.000
sm2: 0.000
sum: 0.116
user: 0.609
card: 0.556
topic: 13.000

1930s                                   TECHNOLOGY          wrong               0.3333333333333333  
qrep: 0.172
skill: 0.035
category: 1.000
leitner: 0.000
sm2: 0.000
sum: 0.342
user: 0.727
card: 0.692
topic: 3.000

_Nash Bridges_                          TELEVISION          wrong               0.25                
qrep: 0.220
skill: 0.032
category: 1.000
leitner: 0.000
sm2: 0.000
sum: 0.344
user: 0.318
card: 0.286
topic: 12.000

king                                    GAMES               correct             0.4                 
qrep: 0.267
skill: 0.182
cat